In [1]:
from transformers import pipeline

ImportError: cannot import name 'pipeline' from 'transformers' (/data/u_kamm_software/EEG data analysis/eeg_llm/.venv/lib/python3.9/site-packages/transformers/__init__.py)